In [24]:
import numpy as np
import pandas as pd
from pathlib import Path
# import os.path
from os import path

from sklearn.model_selection import train_test_split

import tensorflow as tf

In [25]:
image_dir = Path('/kaggle/input/basicshapes/shapes/shapes')

In [26]:
# filepaths = list(image_dir.glob(r'**/*.png'))
# labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))
# filepaths = pd.Series(filepaths, name = 'Filepath').astype(str)
# labels = pd.Series(labels, name = 'Label')
# image_df = pd.concat([filepaths, labels], axis = 1)
# image_df


In [27]:
filepaths = (list(image_dir.glob(r'**/*.png')))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))
df = pd.DataFrame({"Filepath": filepaths, "Label": labels})
df

,Filepath,Label
0,/kaggle/input/basicshapes/shapes/shapes/circle...,circles
1,/kaggle/input/basicshapes/shapes/shapes/circle...,circles
2,/kaggle/input/basicshapes/shapes/shapes/circle...,circles
3,/kaggle/input/basicshapes/shapes/shapes/circle...,circles
4,/kaggle/input/basicshapes/shapes/shapes/circle...,circles
...,...,...
295,/kaggle/input/basicshapes/shapes/shapes/square...,squares
296,/kaggle/input/basicshapes/shapes/shapes/square...,squares
297,/kaggle/input/basicshapes/shapes/shapes/square...,squares
298,/kaggle/input/basicshapes/shapes/shapes/square...,squares


In [28]:
train_df, test_df = train_test_split(image_df, train_size = 0.7, shuffle = True, random_state = 1)
train_df

,Filepath,Label
241,/kaggle/input/basicshapes/shapes/shapes/triang...,triangles
400,/kaggle/input/basicshapes/shapes/triangles/dra...,triangles
286,/kaggle/input/basicshapes/shapes/shapes/triang...,triangles
379,/kaggle/input/basicshapes/shapes/shapes/square...,squares
314,/kaggle/input/basicshapes/shapes/shapes/square...,squares
...,...,...
129,/kaggle/input/basicshapes/shapes/shapes/circle...,circles
144,/kaggle/input/basicshapes/shapes/shapes/circle...,circles
72,/kaggle/input/basicshapes/shapes/circles/drawi...,circles
235,/kaggle/input/basicshapes/shapes/shapes/triang...,triangles


In [29]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

In [30]:
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(28, 28),
    color_mode='grayscale',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(28, 28),
    color_mode='grayscale',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(28, 28),
    color_mode='grayscale',
    class_mode='categorical',
    batch_size=32,
    shuffle=False
)

Found 336 validated image filenames belonging to 3 classes.
Found 84 validated image filenames belonging to 3 classes.
Found 180 validated image filenames belonging to 3 classes.


In [31]:
inputs = tf.keras.Input(shape=(28, 28, 1))
x = tf.keras.layers.Conv2D(filters=6, kernel_size=(3, 3), activation='relu')(inputs)
x = tf.keras.layers.MaxPooling2D()(x)
x = tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPooling2D()(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
outputs = tf.keras.layers.Dense(3, activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint('./model.h5', save_best_only=True, save_weights_only=True)
    ]
)

2022-12-14 17:33:08.474539: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2022-12-14 17:33:08.921420: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100
11/11 [==============================] - 4s 296ms/step - loss: 1.0999 - accuracy: 0.3274 - val_loss: 1.0992 - val_accuracy: 0.2857
Epoch 2/100
11/11 [==============================] - 0s 42ms/step - loss: 1.0989 - accuracy: 0.3363 - val_loss: 1.1012 - val_accuracy: 0.2857
Epoch 3/100
11/11 [==============================] - 1s 46ms/step - loss: 1.0969 - accuracy: 0.3363 - val_loss: 1.0988 - val_accuracy: 0.2857
Epoch 4/100
11/11 [==============================] - 0s 41ms/step - loss: 1.0947 - accuracy: 0.3929 - val_loss: 1.0967 - val_accuracy: 0.3452
Epoch 5/100
11/11 [==============================] - 0s 40ms/step - loss: 1.0930 - accuracy: 0.3542 - val_loss: 1.0927 - val_accuracy: 0.3452
Epoch 6/100
11/11 [==============================] - 0s 45ms/step - loss: 1.0896 - accuracy: 0.3869 - val_loss: 1.0881 - val_accuracy: 0.4762
Epoch 7/100
11/11 [==============================] - 0s 40ms/step - loss: 1.0866 - accuracy: 0.4613 - val_loss: 1.0811 - val_accuracy: 0.4762
Epoch

In [32]:
model.load_weights('./model.h5')

In [35]:
results = model.evaluate(test_images, verbose=0)

print("    Loss: {:.4f}".format(results[0]))
print("Accuracy: {:.2f}%".format(results[1] * 100))

    Loss: 0.5227
Accuracy: 73.89%
